In [6]:
import os
import pandas as pd

# Find file automatically
data_path = ''
for dirname, _, filenames in os.walk('/kaggle/input/covid-19-hospital-capacity'):
    for filename in filenames:
        if filename.endswith('.csv'):
            data_path = os.path.join(dirname, filename)

# Load dataset
df = pd.read_csv(data_path)
print("Loaded:", data_path)
df.head()


Loaded: /kaggle/input/covid-19-hospital-capacity/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State__RAW__20250802.csv


,state,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_anticipated_within_week_yes,critical_staffing_shortage_anticipated_within_week_no,critical_staffing_shortage_anticipated_within_week_not_reported,hospital_onset_covid,hospital_onset_covid_coverage,inpatient_beds,...,previous_day_admission_pediatric_covid_confirmed_5_11_coverage,previous_day_admission_pediatric_covid_confirmed_unknown,previous_day_admission_pediatric_covid_confirmed_unknown_coverage,staffed_icu_pediatric_patients_confirmed_covid,staffed_icu_pediatric_patients_confirmed_covid_coverage,staffed_pediatric_icu_bed_occupancy,staffed_pediatric_icu_bed_occupancy_coverage,total_staffed_pediatric_icu_beds,total_staffed_pediatric_icu_beds_coverage,date
0,AK,1,5,12,1,15,2,0.0,8,1093,...,17,0,18,0,18,5,18,6,18,04/27/2024 12:00:00 AM
1,CA,5,115,217,22,252,63,13.0,158,59666,...,324,0,324,5,329,789,329,1243,329,04/27/2024 12:00:00 AM
2,FL,3,39,162,4,124,76,8.0,101,54450,...,192,1,195,8,198,543,198,777,198,04/27/2024 12:00:00 AM
3,SC,11,8,47,11,38,17,2.0,43,10028,...,59,0,59,0,64,61,64,89,64,04/27/2024 12:00:00 AM
4,MN,0,0,128,6,79,43,0.0,11,9340,...,127,0,127,0,127,94,127,158,127,04/27/2024 12:00:00 AM


In [9]:
# Check for missing values
print(df.isnull().sum())

# Drop rows with missing key values (e.g., utilization metrics)
df.dropna(subset=['inpatient_bed_utilization', 'icu_bed_utilization'], inplace=True)

df = df[(df['fare_amount'] > 0) & (df['fare_amount'] < 1000)]

df = df[(df['pickup_latitude'].between(40, 42)) & 
        (df['pickup_longitude'].between(-75, -72)) &
        (df['dropoff_latitude'].between(40, 42)) &
        (df['dropoff_longitude'].between(-75, -72))]

print("Shape after cleaning:", df.shape)

state                                                     0
critical_staffing_shortage_today_yes                      0
critical_staffing_shortage_today_no                       0
critical_staffing_shortage_today_not_reported             0
critical_staffing_shortage_anticipated_within_week_yes    0
                                                         ..
staffed_pediatric_icu_bed_occupancy                       0
staffed_pediatric_icu_bed_occupancy_coverage              0
total_staffed_pediatric_icu_beds                          0
total_staffed_pediatric_icu_beds_coverage                 0
date                                                      0
Length: 135, dtype: int64


KeyError: ['inpatient_bed_utilization', 'icu_bed_utilization']

In [3]:
# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# Extract temporal features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.day_name()

# Optional: flag pandemic waves (example for March 2020, Jan 2021, etc.)
df['is_wave1'] = df['date'].between('2020-03-01', '2020-05-31').astype(int)
df['is_wave2'] = df['date'].between('2021-01-01', '2021-03-31').astype(int)

# Calculate bed occupancy rate (if columns exist)
if 'inpatient_beds_used_7_day_avg' in df.columns and 'total_beds_7_day_avg' in df.columns:
    df['bed_occupancy_rate'] = (
        df['inpatient_beds_used_7_day_avg'] / df['total_beds_7_day_avg']
    ).round(2)

# Show result
cols_to_display = ['date', 'year', 'month', 'weekday', 'is_wave1', 'is_wave2']
if 'bed_occupancy_rate' in df.columns:
    cols_to_display.append('bed_occupancy_rate')

print(df[cols_to_display].head())


        date  year  month   weekday  is_wave1  is_wave2
0 2024-04-27  2024      4  Saturday         0         0
1 2024-04-27  2024      4  Saturday         0         0
2 2024-04-27  2024      4  Saturday         0         0
3 2024-04-27  2024      4  Saturday         0         0
4 2024-04-27  2024      4  Saturday         0         0


In [5]:
# Export cleaned COVID-19 hospital dataset
df.to_csv('/kaggle/working/covid_cleaned_dataset.csv', index=False)
print("Cleaned dataset saved as covid_cleaned_dataset.csv")


Cleaned dataset saved as covid_cleaned_dataset.csv
